In [24]:
from TSPLIPLoader import TPSLIBLoader

berlin52 = TPSLIBLoader('berlin52')
burma14 = TPSLIBLoader('burma14')


berlin52.nodes

best_cost = berlin52.get_solution

In [25]:
berlin52.get_solution

7542

In [26]:
from HSFFPKO import HSFFPKO, universal_tsp_wrapper
from EHO import elephant_herd_optimization
from ACO import ant_colony
from SA import simulated_annealing
from SAI import simulated_annealing_improved
from GWO import GreyWolfTSP
from PSO import hybrid_pso_tsp
from GA import genetic_alg 

num_nodes = len(berlin52.nodes)
graph = berlin52.to_graph()
matrix = berlin52.to_matrix()

In [27]:
import time 
import numpy as np

data = {
    'HSFFPKO': [],
    'EHO': [],
    'ACO': [],
    'SA' : [],
    'SAI': [],
    'GWO': [],
    # 'PSO': [],
    'GA': []
    
}

N = 10
for _ in range(N):
    start = time.perf_counter()
    output = HSFFPKO(
        Popsize=50,
        Maxiteration=500,
        LB=np.zeros(num_nodes),
        UB=np.ones(num_nodes) * 20,
        Dim=num_nodes,
        Fobj=universal_tsp_wrapper(matrix),  # MATRIX
        scouts_rate=0.15,
        num_flocks=8
    )[0] 
    end = time.perf_counter()
    data['HSFFPKO'].append([output,end - start])
    
    start = time.perf_counter()
    output = elephant_herd_optimization(graph)[1]
    end = time.perf_counter()
    data['EHO'].append([output,end - start])

    start = time.perf_counter()
    output = ant_colony(graph)[1]
    end = time.perf_counter()
    data['ACO'].append([output,end - start])

    start = time.perf_counter()
    output = simulated_annealing(graph, temperature=1000, alpha=0.995, iterations=1000)[1]
    end = time.perf_counter()
    data['SA'].append([output,end - start])

    start = time.perf_counter()
    output = simulated_annealing_improved(graph, iter_per_temp=100)[0]
    end = time.perf_counter()
    data['SAI'].append([output,end - start])

    gwo = GreyWolfTSP(graph)
    start = time.perf_counter()
    output = gwo.optimize()[1]
    end = time.perf_counter()
    data['GWO'].append([output,end - start])

    # start = time.perf_counter()
    # output = hybrid_pso_tsp(graph)[1]
    # end = time.perf_counter()
    # data['PSO'].append([output,end - start])

    start = time.perf_counter()
    output = genetic_alg(graph, pop_size=100, mutation_rate=0.01, generations=50)[1]
    end = time.perf_counter()
    data['GA'].append([output,end - start])




In [30]:
import pandas as pd

cost_dict = {k: [x[0] for x in v] for k, v in data.items()}
df_cost = pd.DataFrame(cost_dict)

runtime_dict = {k: [x[1] for x in v] for k, v in data.items()}
df_runtime = pd.DataFrame(runtime_dict)

# Get the relative error
def relative_error(costs, best_cost):
    return (costs - best_cost) / best_cost

rel_error_dict = {k: relative_error(np.array(v), best_cost) for k, v in cost_dict.items()}
df_rel_error = pd.DataFrame(rel_error_dict)

cols = ['HSFFPKO', 'EHO', 'ACO', 'SA', 'SAI', 'GWO', 'GA']

df = pd.DataFrame(
    np.vstack([df_cost.mean(), df_runtime.mean(), df_cost.min(), df_rel_error.mean() * 100, df_rel_error.min() * 100]),
    columns=cols,
    index=['Avg Cost', 'Avg Runtime', 'Min Cost', 'Avg Rel Error', 'Min Rel Error']
).transpose()

df.sort_values(by='Avg Cost', inplace=True)

df

,Avg Cost,Avg Runtime,Min Cost,Avg Rel Error,Min Rel Error
GA,7542.0,185.172610,7542.0,0.000000,0.000000
ACO,7708.4,29.309873,7649.0,2.206311,1.418722
SAI,7925.4,2.653776,7542.0,5.083532,0.000000
EHO,9269.2,0.502292,8919.0,22.901087,18.257757
SA,14025.1,0.011572,13342.0,85.959958,76.902678
GWO,15071.1,1.830700,12725.0,99.828958,68.721824
HSFFPKO,16341.5,1.840032,14427.0,116.673296,91.288783


In [31]:
df_cost

,HSFFPKO,EHO,ACO,SA,SAI,GWO,GA
0,15572.0,9073.0,7679.0,14198,8095,18067,7542
1,16567.0,9446.0,7663.0,14214,7542,15299,7542
2,17534.0,8944.0,7679.0,13342,8270,18908,7542
3,17499.0,9251.0,7649.0,13677,7799,13779,7542
4,16528.0,9433.0,7679.0,15310,8046,15361,7542
5,16531.0,9863.0,7679.0,13705,7542,12725,7542
6,16717.0,8942.0,7703.0,14350,7715,16203,7542
7,15550.0,9113.0,7777.0,13780,8262,14758,7542
8,16490.0,8919.0,7785.0,13900,8053,12744,7542
9,14427.0,9708.0,7791.0,13775,7930,12867,7542


In [32]:
df_runtime

,HSFFPKO,EHO,ACO,SA,SAI,GWO,GA
0,1.964209,0.505576,28.507050,0.012632,2.581066,1.711579,173.480670
1,1.883073,0.507690,28.875904,0.011879,2.625969,1.859419,174.134228
2,2.005269,0.546509,30.258004,0.011295,2.598887,1.827647,195.644245
3,1.702085,0.469091,28.111425,0.010889,2.495967,1.718937,157.955317
4,1.743511,0.483594,29.487353,0.011575,2.774755,1.991701,191.748684
5,1.748167,0.490397,29.312589,0.011151,2.604696,1.846779,154.837591
6,1.932780,0.529498,30.217396,0.011665,2.638243,1.728166,179.981497
7,1.758613,0.497810,29.404076,0.011565,2.718445,1.848509,254.458546
8,1.917038,0.513760,29.850772,0.011645,2.716515,1.838027,175.723837
9,1.745577,0.478996,29.074163,0.011427,2.783212,1.936237,193.761486
